
# Analysis Comparison between Original and Expanded Lexicons on Cohort Analysis Task

Subreddit: PTSD

In [1]:
from pathlib import Path
import pandas as pd

from config import global_config
from lexicons2 import Values, ValuesExpanded, Liwc2015, Liwc2015Expanded
from spaces import WordEmbeddings, LabelEmbeddings

In [2]:
COHORT_NAME = 'ptsd'

In [3]:
cohort_corpus_path = global_config.reddit.submissions[COHORT_NAME]
cohort_models_path = f'{global_config.paths.models}/{COHORT_NAME}'

control_corpus_path = global_config.reddit.submissions['IAMA']
control_models_path = f'{global_config.paths.models}/IAMA'

In [4]:
cohort_ranks_csv = f'{global_config.paths.ranks}/{COHORT_NAME}_ranks.csv'
expanded_cohort_ranks_csv = f'{global_config.paths.ranks}/{COHORT_NAME}_ranks_expanded.csv'

control_ranks_csv = f'{global_config.paths.ranks}/IAMA_ranks.csv'
expanded_control_ranks_csv = f'{global_config.paths.ranks}/IAMA_ranks_expanded.csv'

relative_ranks_csv = f'{global_config.paths.ranks}/{COHORT_NAME}_relative-ranks.csv'
expanded_relative_ranks_csv = f'{global_config.paths.ranks}/{COHORT_NAME}_relative-ranks_expanded.csv'

Build Control Spaces

In [5]:
control_word_space = WordEmbeddings(corpus_path=control_corpus_path, model_path=control_models_path).build()

In [6]:
org_control_label_space = LabelEmbeddings(lexicons=[Liwc2015(), Values()], word_embeddings=control_word_space).build()

In [7]:
expanded_control_label_space = LabelEmbeddings(lexicons=[Liwc2015Expanded(), ValuesExpanded()], word_embeddings=control_word_space).build()

Build Cohort Spaces

In [8]:
cohort_word_space = WordEmbeddings(corpus_path=cohort_corpus_path, model_path=cohort_models_path).build()

In [9]:
org_cohort_label_space = LabelEmbeddings(lexicons=[Liwc2015(), Values()], word_embeddings=cohort_word_space).build()

In [10]:
expanded_cohort_label_space = LabelEmbeddings(lexicons=[Liwc2015Expanded(), ValuesExpanded()], word_embeddings=cohort_word_space).build()

Save and view results

In [11]:
def save_if_not_save_and_return(space: LabelEmbeddings, path: str):
	if not Path(path).exists():
		space.save_distances_to_csv(path)
	return pd.read_csv(path, names=['label_one', 'label_two', 'distance'])

control_ranks = save_if_not_save_and_return(org_control_label_space, control_ranks_csv)
expanded_control_ranks = save_if_not_save_and_return(expanded_control_label_space, expanded_control_ranks_csv)

cohort_ranks = save_if_not_save_and_return(org_cohort_label_space, cohort_ranks_csv)
expanded_cohort_ranks = save_if_not_save_and_return(expanded_cohort_label_space, expanded_cohort_ranks_csv)

In [13]:
org_cohort_label_space.compute_rank_deltas(org_control_label_space, relative_ranks_csv)
relative_ranks = pd.read_csv(relative_ranks_csv, names=['label_one', 'label_two', 'current_rank', 'control_rank', 'current_distance', 'control_distance', 'rank_delta', 'distance_delta'])

expanded_cohort_label_space.compute_rank_deltas(expanded_control_label_space, expanded_relative_ranks_csv)
expanded_relative_ranks = pd.read_csv(expanded_relative_ranks_csv, names=['label_one', 'label_two', 'current_rank', 'control_rank', 'current_distance', 'control_distance', 'rank_delta', 'distance_delta'])

View Stuff

In [14]:
relative_ranks

,label_one,label_two,current_rank,control_rank,current_distance,control_distance,rank_delta,distance_delta
0,liwc2015:function,values:learning,67,28,0.151187,0.178845,39,-0.027657
1,liwc2015:function,values:feeling-good,81,43,0.733822,0.499940,38,0.233882
2,liwc2015:function,liwc2015:money,77,51,0.349224,0.681783,26,-0.332559
3,liwc2015:function,liwc2015:work,56,36,0.070017,0.336770,20,-0.266753
4,liwc2015:function,liwc2015:netspeak,82,62,0.901148,0.781215,20,0.119934
...,...,...,...,...,...,...,...,...
6967,values:animals,liwc2015:swear,1,59,0.635042,0.909386,-58,-0.274344
6968,values:animals,liwc2015:sexual,5,75,0.678243,1.021016,-70,-0.342773
6969,values:animals,values:parents,7,78,0.692603,1.069746,-71,-0.377143
6970,values:animals,values:truth,0,71,0.634491,0.982374,-71,-0.347884


In [15]:
expanded_relative_ranks

,label_one,label_two,current_rank,control_rank,current_distance,control_distance,rank_delta,distance_delta
0,liwc2015:function,liwc2015:they,44,19,0.013176,0.103689,25,-0.090512
1,liwc2015:function,liwc2015:work,64,39,0.070017,0.336770,25,-0.266753
2,liwc2015:function,liwc2015:money,80,58,0.349224,0.681783,22,-0.332559
3,liwc2015:function,liwc2015:posemo,52,34,0.023454,0.259349,18,-0.235894
4,liwc2015:function,liwc2015:netspeak,83,68,0.901148,0.781215,15,0.119934
...,...,...,...,...,...,...,...,...
7135,liwc2015:filler,liwc2015:cogproc,20,57,0.309082,0.998537,-37,-0.689455
7136,liwc2015:filler,liwc2015:power,6,43,0.306198,0.982761,-37,-0.676563
7137,liwc2015:filler,liwc2015:adj,11,49,0.308220,0.988968,-38,-0.680748
7138,liwc2015:filler,liwc2015:compare,4,45,0.305423,0.984765,-41,-0.679341
